In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
import csv
import glob
import pickle

In [20]:
folder_path = 'Data/Next/*.csv'
files = glob.glob(folder_path)

df = pd.concat([pd.read_csv(file, index_col=None) for file in files], axis=0, ignore_index=True)

In [21]:
data_inputs = np.asarray(df.iloc[:,0:6])

label_encoder = preprocessing.LabelEncoder()
data_labels = np.expand_dims(np.array(label_encoder.fit_transform(data.iloc[:,6])),axis=1)

fileob = open('pickled_label_encoder.obj','wb')
pickle.dump(label_encoder,fileob)
fileob.close()

data_array = np.concatenate((data_inputs,data_labels), axis=1)

In [32]:
def window_dataset(data,window_size,shift):
    
    num_windows = int(np.floor((len(data)-window_size)/shift))
    windowed_data = np.zeros([num_windows,window_size,np.shape(data)[1]])

    for i in range(num_windows):
        windowed_data[i,:,:] = data[i*shift:i*shift+window_size,:]
    
    return windowed_data

In [33]:
window_size = 40
shift = 20
windowed_data = window_dataset(data_array,window_size,shift)

shuffled_data = np.take(windowed_data,np.random.rand(windowed_data.shape[0]).argsort(),axis=0)

window_labels = shuffled_data[:,:,6]
mask = np.mean(window_labels, axis=1) == np.min(window_labels, axis=1)

labels = np.mean(window_labels, axis=1)[mask].astype(np.int)

input_data_1 = np.array(shuffled_data[mask,:,0])
input_data_2 = np.array(shuffled_data[mask,:,1])
input_data_3 = np.array(shuffled_data[mask,:,2])
input_data_4 = np.array(shuffled_data[mask,:,3])
input_data_5 = np.array(shuffled_data[mask,:,4])
input_data_6 = np.array(shuffled_data[mask,:,5])
print(input_data_1.shape)

(409, 40)


In [34]:
input_1 = tf.keras.layers.Input(shape=[window_size,1])
conv_1 = tf.keras.layers.Conv1D(5, 5, activation='relu')(input_1)
input_2 = tf.keras.layers.Input(shape=[window_size,1])
conv_2 = tf.keras.layers.Conv1D(5, 5, activation='relu')(input_2)
input_3 = tf.keras.layers.Input(shape=[window_size,1])
conv_3 = tf.keras.layers.Conv1D(5, 5, activation='relu')(input_3)
input_4 = tf.keras.layers.Input(shape=[window_size,1])
conv_4 = tf.keras.layers.Conv1D(5, 5, activation='relu')(input_4)
input_5 = tf.keras.layers.Input(shape=[window_size,1])
conv_5 = tf.keras.layers.Conv1D(5, 5, activation='relu')(input_5)
input_6 = tf.keras.layers.Input(shape=[window_size,1])
conv_6 = tf.keras.layers.Conv1D(5, 5, activation='relu')(input_6)

concat = tf.keras.layers.Concatenate()([conv_1, conv_2, conv_3, conv_4, conv_5, conv_6])
flatten = tf.keras.layers.Flatten()(concat)
dense1 = tf.keras.layers.Dense(30, activation='relu')(flatten)
dense2 = tf.keras.layers.Dense(30, activation='relu')(dense1)
output = tf.keras.layers.Dense(np.max(labels)+1, activation = 'softmax')(dense2)
model = tf.keras.Model(inputs=[input_1, input_2, input_3, input_4, input_5, input_6], outputs=[output])

In [35]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 1)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 40, 1)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 40, 1)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 40, 1)]      0                                            
_______________________________________________________________________________________

In [36]:
dummy_labels = tf.keras.utils.to_categorical(labels)

In [37]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit([input_data_1,input_data_2,input_data_3,
                     input_data_4,input_data_5,input_data_6],dummy_labels, epochs = 50,validation_split=0.15)

Epoch 1/50
11/11 [==============================] - 0s 27ms/step - loss: 2.5564 - accuracy: 0.3573 - val_loss: 1.4019 - val_accuracy: 0.5161
Epoch 2/50
11/11 [==============================] - 0s 14ms/step - loss: 0.6387 - accuracy: 0.7233 - val_loss: 0.9818 - val_accuracy: 0.6935
Epoch 3/50
11/11 [==============================] - 0s 6ms/step - loss: 0.3531 - accuracy: 0.8357 - val_loss: 0.8593 - val_accuracy: 0.7742
Epoch 4/50
11/11 [==============================] - 0s 6ms/step - loss: 0.2242 - accuracy: 0.9625 - val_loss: 0.5635 - val_accuracy: 0.8065
Epoch 5/50
11/11 [==============================] - 0s 6ms/step - loss: 0.1452 - accuracy: 0.9856 - val_loss: 0.5359 - val_accuracy: 0.8226
Epoch 6/50
11/11 [==============================] - 0s 6ms/step - loss: 0.0880 - accuracy: 0.9942 - val_loss: 0.4746 - val_accuracy: 0.8387
Epoch 7/50
11/11 [==============================] - 0s 6ms/step - loss: 0.0484 - accuracy: 0.9942 - val_loss: 0.4872 - val_accuracy: 0.8710
Epoch 8/50
11/11 [

In [38]:
model.save('HAR_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: HAR_model/assets
